In [1]:
import pandas as pd
import tldextract

# -----------------------------------------
# 1) Cargar dataset base
# -----------------------------------------
PATH = "../data/clean/dataset_base_v21.csv"
df = pd.read_csv(PATH).copy()

df.head()


,campaign,categoria,confianza,confidence,dataset_split,domain,entidad,free_hosting,inclusion,is_https,...,url_norm,domain_complexity,host_entropy,domain_whitelist_score,suspicious_path_token,token_density,trusted_token_context,infra_risk,fake_tld_in_subdomain_or_path,param_count_boost
0,NaN,banca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://www.caixabank.es/particular/banca-digi...,0.508495,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,95.0,NaN,NaN,Ibercaja,0.0,NaN,1.0,...,https://www.ibercaja.es/particulares,0.501250,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,90.0,NaN,NaN,WiZink,0.0,NaN,1.0,...,https://www.wizink.es/tarjetas,0.451477,0.0,0.0,0.0,0.500,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,95.0,NaN,NaN,WiZink,0.0,NaN,1.0,...,https://www.wizink.es,0.451477,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,90.0,NaN,NaN,Cetelem,0.0,NaN,1.0,...,https://www.cetelem.es/credito-y-prestamos,0.456514,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd
import tldextract

# -----------------------------------------
# 1) Cargar dataset base
# -----------------------------------------
PATH = "../data/clean/dataset_base_v21.csv"
df = pd.read_csv(PATH).copy()

df.head()


In [3]:
# -----------------------------------------
# 2) Cargar WHITELIST oficial (ES + global neutral)
# -----------------------------------------
# Ajusta la ruta si tu whitelist está en otro sitio

WHITELIST_PATH = "../docs//whitelist.csv"
whitelist_df = pd.read_csv(WHITELIST_PATH)

# Normalizamos a minúsculas
whitelist = set(whitelist_df["domain"].str.lower().tolist())

len(whitelist), list(whitelist)[:10]


(300,
 ['vinted.es',
  'bankinter.com',
  'telecinco.es',
  'gstatic.com',
  'elsevier.es',
  'hetzner.com',
  'googleusercontent.com',
  'dropbox.com',
  'santander.com',
  'prensaiberica.es'])

In [4]:
# -----------------------------------------
# 3) Construir brands_set a partir de WHITELIST
# -----------------------------------------

brands_set = { domain.split(".")[0] for domain in whitelist }

print("Número de marcas detectadas:", len(brands_set))
print("Ejemplos:", list(brands_set)[:15])


Número de marcas detectadas: 291
Ejemplos: ['uniovi', 'cruzroja', 'decathlon', 'gmail', 'blogs', 'elsevier', 'linode', 'socy', 'csic', 'mi-dominio', 'instyle', 'unizar', 'kutxabank', 'fastlylb', 'linguee']


In [5]:
# -----------------------------------------
# 4) Función brand_match_flag_v2
# -----------------------------------------

def compute_brand_match_flag(url):
    ext = tldextract.extract(url)
    core = ext.domain.lower()  # Núcleo del dominio
    
    # Coincidencia EXACTA
    return int(core in brands_set)

df["brand_match_flag_v2"] = df["url"].apply(compute_brand_match_flag)


In [6]:
# -----------------------------------------
# 5) Métricas por clase
# -----------------------------------------

print("Media por clase (label=0 legítimas, label=1 phishing):")
display(df.groupby("label")["brand_match_flag_v2"].mean())


Media por clase (label=0 legítimas, label=1 phishing):


label
0    0.729508
1    0.036290
Name: brand_match_flag_v2, dtype: float64

In [8]:
df[(df["label"] == 1) & (df["brand_match_flag_v2"] == 1)][["url","domain"]].head(20)


,url,domain
371,https://serviciodecorreo.es/,NaN
372,https://sites.google.com/view/habilitaciondeac...,NaN
373,https://sites.google.com/view/serviciosexclusi...,NaN
397,https://misaki4552.github.io/Pagina-instagram/,NaN
401,https://sites.google.com/view/confirmaciondeda...,NaN
413,https://sites.google.com/view/ruraless/p%C3%A1...,NaN
424,http://www.habbocreditosparati.blogspot.com/,NaN
436,https://portaldigitalclientefeliz.blogspot.com...,NaN
461,http://unicismadrid.es/wp-content/com/index/ch...,NaN


In [9]:
df[df["label"] == 1][["url","domain","brand_match_flag_v2"]].head(40)


,url,domain,brand_match_flag_v2
244,https://caixabank-es-883f1e.ingress-erytho.eas...,caixabank-es-883f1e.ingress-erytho.easywp.com,0
245,http://122.114.173.242:30/bancosantander.es/pa...,122.114.173.242:30,0
246,http://wingerdgastehuis.co.za/Bienvenido%20a%2...,wingerdgastehuis.co.za,0
247,http://webseguridadcuenta-9e626b.ingress-bonde...,webseguridadcuenta-9e626b.ingress-bonde.easywp...,0
248,https://www.mobile.kinman.com/.well-known/acme...,www.mobile.kinman.com,0
249,https://wififpt.com.vn/es/bankia.es/es/acceso-...,wififpt.com.vn,0
250,https://web-5.builderallwppro.com/necorreos/vv...,web-5.builderallwppro.com,0
251,https://suponsoro22-ba9799.ingress-daribow.ewp...,suponsoro22-ba9799.ingress-daribow.ewp.live,0
252,https://suponsoo22-ba6aa2.ingress-florina.ewp....,suponsoo22-ba6aa2.ingress-florina.ewp.live,0
253,https://supoertas22-bb468f.ingress-bonde.ewp.l...,supoertas22-bb468f.ingress-bonde.ewp.live,0
